In [1]:
import os
import pandas
from sklearn.feature_extraction.text import CountVectorizer
import json
import re
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

text_stemmer = PorterStemmer()
count_vectorizer = CountVectorizer(max_features=5000, stop_words="english")

In [2]:
movies = pandas.read_csv('movies-archive-data/tmdb_5000_movies.csv')
credits = pandas.read_csv('movies-archive-data/tmdb_5000_credits.csv')

In [3]:
movies = movies.merge(credits, on='title')

In [4]:
movies.shape

(4809, 23)

In [5]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [6]:
movies.dropna(inplace=True)

In [7]:
 movies.shape

(4806, 7)

In [8]:
movies.duplicated().sum()

0

In [9]:
def normalize_column(json_data, column_key, item_count=-1):
    my_obj = json.loads(json_data)
    end_index = len(my_obj) if item_count <= 0 else item_count
    data = [re.sub(r'\s+', '', item[column_key]) for item in my_obj[:end_index]]
    return data


def pluck_director(json_data):
    my_obj = json.loads(json_data)
    director = [re.sub(r'\s+', '', item["name"]) for item in my_obj if item["job"] == "Director"]
    return director


def stem_text(text):
    return " ".join(map(text_stemmer.stem, text.split()))

In [10]:
# normalize these objects from objects to lists of plucked values
movies["genres"] = movies["genres"].apply(lambda json_data: normalize_column(json_data, "name"))
movies["keywords"] = movies["keywords"].apply(lambda json_data: normalize_column(json_data, "name"))
movies["cast"] = movies["cast"].apply(lambda json_data: normalize_column(json_data, "name", 3))
movies["crew"] = movies["crew"].apply(pluck_director)
movies["overview"] = movies["overview"].apply(lambda str_data: str_data.split())

# classify the data
training_data = movies[["movie_id", "title"]]
training_data["stemmed"] = (
        movies["overview"] + movies["genres"] + movies["keywords"] + movies["cast"] + movies["crew"]
).apply(lambda str_text: " ".join(str_text).lower())
training_data["stemmed"] = training_data["stemmed"].apply(stem_text)
similarity_vectors = count_vectorizer.fit_transform(training_data["stemmed"]).toarray()
similarity_scores = cosine_similarity(similarity_vectors)

C:\Users\thisi\AppData\Local\Temp\ipykernel_19516\1493184996.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data["stemmed"] = (
C:\Users\thisi\AppData\Local\Temp\ipykernel_19516\1493184996.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data["stemmed"] = training_data["stemmed"].apply(stem_text)


In [13]:
    def get_recommendations(movie_title):
        try:
            index = training_data[training_data["title"] == movie_title].index[0]
            distances = sorted(list(enumerate(similarity_scores[index])), reverse=True, key=lambda x: x[1])

            # Using list comprehension to collect recommendations
            recommendations = [
                {
                    'title': training_data.iloc[i[0]]['title'],
                    'movie_id': int(training_data.iloc[i[0]]['movie_id'])
                }
                for i in distances[1:6]
            ]
            return recommendations
        except IndexError:
            return []

In [14]:
get_recommendations('Spider-Man')

[{'title': 'Spider-Man 3', 'movie_id': 559},
 {'title': 'Spider-Man 2', 'movie_id': 558},
 {'title': 'The Amazing Spider-Man 2', 'movie_id': 102382},
 {'title': 'Arachnophobia', 'movie_id': 6488},
 {'title': 'Kick-Ass', 'movie_id': 23483}]

In [15]:
get_recommendations('Batman')

[{'title': 'Batman', 'movie_id': 2661},
 {'title': 'Batman & Robin', 'movie_id': 415},
 {'title': 'Batman Begins', 'movie_id': 272},
 {'title': 'Batman Returns', 'movie_id': 364},
 {'title': 'The R.M.', 'movie_id': 36825}]